<a href="https://colab.research.google.com/github/sindhsandhiya201999/-mcis6273-s25-datamining/blob/main/Sandhiyahw0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/sindhsandhiya201999/mcis6273_s25_datamining.git


Cloning into 'mcis6273_s25_datamining'...
remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 21 (delta 6), reused 21 (delta 6), pack-reused 0 (from 0)
Receiving objects: 100% (21/21), 136.41 KiB | 1.84 MiB/s, done.
Resolving deltas: 100% (6/6), done.


In [2]:
%cd mcis6273_s25_datamining


/content/mcis6273_s25_datamining


# **Download and Load the Dataset**

In [3]:
!wget -P data/ https://raw.githubusercontent.com/kmsaumcis/mcis6273_s25_datamining/main/hw0/data/2024_flavors_of_cacoa.tsv


--2025-02-19 05:26:47--  https://raw.githubusercontent.com/kmsaumcis/mcis6273_s25_datamining/main/hw0/data/2024_flavors_of_cacoa.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 272727 (266K) [text/plain]
Saving to: ‘data/2024_flavors_of_cacoa.tsv’

2024_flavors_of_cac 100%[===================>] 266.33K  --.-KB/s    in 0.04s   

2025-02-19 05:26:47 (7.06 MB/s) - ‘data/2024_flavors_of_cacoa.tsv’ saved [272727/272727]



In [4]:
import pandas as pd

df = pd.read_csv("./data/2024_flavors_of_cacoa.tsv", sep='\t')
df.head()


,REF,Company (Manufacturer),Company Location,Review Date,Country of Bean Origin,Specific Bean Origin or Bar Name,Cocoa Percent,Ingredients,Most Memorable Characteristics,Rating
0,2454,5150,U.S.A.,2019,Madagascar,"Bejofo Estate, batch 1",76%,"3- B,S,C","cocoa, blackberry, full body",3.75
1,2458,5150,U.S.A.,2019,Dominican Republic,"Zorzal, batch 1",76%,"3- B,S,C","cocoa, vegetal, savory",3.50
2,2454,5150,U.S.A.,2019,Tanzania,"Kokoa Kamili, batch 1",76%,"3- B,S,C","rich cocoa, fatty, bready",3.25
3,2542,5150,U.S.A.,2021,India,"Anamalai, batch 1",68%,"3- B,S,C","milk brownie, macadamia,chewy",3.50
4,2546,5150,U.S.A.,2021,Uganda,"Semuliki Forest, batch 1",80%,"3- B,S,C","mildly bitter, basic cocoa, fatty",3.25


# **Data Cleaning**

In [12]:
df["Cocoa Percent"] = df["Cocoa Percent"].astype(str).str.replace('%', '').astype(float) / 100


In [17]:
df["Ingredients"] = df["Ingredients"].astype(str).fillna("0-")



In [18]:
df["ingredient_count"] = df["Ingredients"].apply(lambda x: x.split("-")[0].strip() if "-" in x else "0")


In [19]:
df["ingredient_count"] = pd.to_numeric(df["ingredient_count"], errors="coerce").fillna(0).astype(int)


In [20]:
df["ingredients"] = df["Ingredients"].apply(lambda x: x.split("-")[1].strip().split(",") if "-" in x else [])


In [21]:
ingredient_types = ["B", "C", "L", "S", "S*", "Sa", "V"]
for ing in ingredient_types:
    df[ing] = df["ingredients"].apply(lambda x: 1 if isinstance(x, list) and ing in x else 0)

In [22]:
df.drop(columns=["Ingredients", "ingredients"], inplace=True)


In [26]:
df["Most Memorable Characteristics"] = df["Most Memorable Characteristics"].astype(str).fillna("")

# Step 10: Create dummy variables for characteristics
characteristics = df["Most Memorable Characteristics"].str.get_dummies(sep=", ")

# Step 11: Keep only characteristics that appear at least 20 times
frequent_characteristics = characteristics.loc[:, characteristics.sum() >= 20]

# Step 12: Merge the new characteristic columns back into the main DataFrame
df = pd.concat([df, frequent_characteristics], axis=1)

# Step 13: Drop the original "Most Memorable Characteristics" column
df.drop(columns=["Most Memorable Characteristics"], inplace=True)

# Step 14: Save the cleaned dataset
df.to_csv("./data/cleaned_data_2025_flavors_of_cacao.csv", index=False)

# Step 15: Display first 5 rows to verify
df.head()

,REF,Company (Manufacturer),Company Location,Review Date,Country of Bean Origin,Specific Bean Origin or Bar Name,Cocoa Percent,Rating,ingredient_count,B,...,spicy,sticky,strawberry,sweet,tangy,tart,tobacco,vanilla,vegetal,woody
0,2454,5150,U.S.A.,2019,Madagascar,"Bejofo Estate, batch 1",7.600000e-07,3.75,3,1,...,0,0,0,0,0,0,0,0,0,0
1,2458,5150,U.S.A.,2019,Dominican Republic,"Zorzal, batch 1",7.600000e-07,3.50,3,1,...,0,0,0,0,0,0,0,0,1,0
2,2454,5150,U.S.A.,2019,Tanzania,"Kokoa Kamili, batch 1",7.600000e-07,3.25,3,1,...,0,0,0,0,0,0,0,0,0,0
3,2542,5150,U.S.A.,2021,India,"Anamalai, batch 1",6.800000e-07,3.50,3,1,...,0,0,0,0,0,0,0,0,0,0
4,2546,5150,U.S.A.,2021,Uganda,"Semuliki Forest, batch 1",8.000000e-07,3.25,3,1,...,0,0,0,0,0,0,0,0,0,0


In [28]:
# Load the cleaned dataset
df = pd.read_csv("./data/cleaned_data_2025_flavors_of_cacao.csv")

# Step 1: Create a reduced dataset with selected columns
df_reduced = df[["Review Date", "Country of Bean Origin", "Cocoa Percent", "Rating"]]

# Step 2: Save the reduced dataset as CSV and JSON
df_reduced.to_csv("./data/data_reduced_2025_flavors_of_cacao.csv", index=False)
df_reduced.to_json("./data/data_reduced_2025_flavors_of_cacao.json", orient="records")

# Step 3: Apply filters to create a filtered dataset
df_filtered = df[
    (df["Rating"] >= 3.25) &  # Chocolates with rating 3.25+
    (df["Cocoa Percent"].between(0.65, 0.75)) &  # Cocoa content between 65% and 75%
    (df["Review Date"].between(2018, 2021)) &  # Reviewed between 2018 and 2021
    (df[["fatty", "earthy", "roasty"]].sum(axis=1) > 0)  # Has at least one of these characteristics
]

# Step 4: Save the filtered dataset as CSV and JSON
df_filtered.to_csv("./data/data_filtered_2025_flavors_of_cacao.csv", index=False)
df_filtered.to_json("./data/data_filtered_2025_flavors_of_cacao.json", orient="records")


In [29]:
import os

# List all files in the /data/ folder
os.listdir("./data/")


['data_reduced_2025_flavors_of_cacao.json',
 'data_reduced_2025_flavors_of_cacao.csv',
 'data_filtered_2025_flavors_of_cacao.json',
 'data_filtered_2025_flavors_of_cacao.csv',
 '2024_flavors_of_cacoa.tsv',
 'cleaned_data_2025_flavors_of_cacao.csv']

In [30]:
mkdir -p homework/hw0


In [34]:

# Move all necessary files to homework/hw0/
!mv ./data/cleaned_data_2025_flavors_of_cacao.csv homework/hw0/
!mv ./data/data_reduced_2025_flavors_of_cacao.csv homework/hw0/
!mv ./data/data_reduced_2025_flavors_of_cacao.json homework/hw0/
!mv ./data/data_filtered_2025_flavors_of_cacao.csv homework/hw0/
!mv ./data/data_filtered_2025_flavors_of_cacao.json homework/hw0/
!mv ./Sandhiyahw0.ipynb homework/hw0/  # Ensure this is the correct notebook filename

# Zip the homework directory
!zip -r Sandhiya_hw0_files.zip homework/hw0/

# Download the ZIP file
from google.colab import files
files.download("Sandhiya_hw0_files.zip")


mv: cannot stat './Sandhiyahw0.ipynb': No such file or directory
  adding: homework/hw0/ (stored 0%)
  adding: homework/hw0/data_reduced_2025_flavors_of_cacao.json (deflated 95%)
  adding: homework/hw0/data_reduced_2025_flavors_of_cacao.csv (deflated 86%)
  adding: homework/hw0/data_filtered_2025_flavors_of_cacao.json (stored 0%)
  adding: homework/hw0/data_filtered_2025_flavors_of_cacao.csv (deflated 42%)
  adding: homework/hw0/cleaned_data_2025_flavors_of_cacao.csv (deflated 89%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [35]:
!ls *.ipynb


ls: cannot access '*.ipynb': No such file or directory


In [37]:
!find / -name "*.ipynb" 2>/dev/null


/usr/local/lib/python3.11/dist-packages/notebook/bundler/tests/resources/empty.ipynb
/usr/local/lib/python3.11/dist-packages/panel/tests/ui/io/app.ipynb
/usr/local/lib/python3.11/dist-packages/nbclassic/bundler/tests/resources/empty.ipynb
/usr/local/lib/python3.11/dist-packages/flax/core/flax_functional_engine.ipynb
/usr/local/lib/python3.11/dist-packages/wasabi/tests/test-data/wasabi-test-notebook.ipynb
/usr/local/lib/python3.11/dist-packages/holoviews/tests/ipython/notebooks/test_output_svg_line_magic.ipynb
/usr/local/lib/python3.11/dist-packages/holoviews/tests/ipython/notebooks/test_opts_image_cell_magic.ipynb
/usr/local/lib/python3.11/dist-packages/holoviews/tests/ipython/notebooks/test_opts_image_line_magic.ipynb
/usr/local/lib/python3.11/dist-packages/holoviews/tests/ipython/notebooks/test_opts_image_cell_magic_offset.ipynb
/usr/local/lib/python3.11/dist-packages/holoviews/examples/getting_started/5-Live_Data.ipynb
/usr/local/lib/python3.11/dist-packages/holoviews/examples/getti

In [38]:
!mv /content/drive/MyDrive/Sandhiyahw0.ipynb homework/hw0/


mv: cannot stat '/content/drive/MyDrive/Sandhiyahw0.ipynb': No such file or directory
